In [1]:
from bs4 import BeautifulSoup
from bs4 import ResultSet
import numpy as np
import pandas as pd
import requests
import re
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
# A dictionary consisting of links from which the data would be gathered
urls = {
    'ogloszenia_wtórny': 'https://ogloszenia.trojmiasto.pl/nieruchomosci-rynek-wtorny/f1i,1_2_3,ri,1_,wi,100_200_230_250_260_220_240_210,xi,1900_.html',
    'ogloszenia_pierwotny': 'https://ogloszenia.trojmiasto.pl/nieruchomosci-rynek-pierwotny/f1i,1_2_3,ri,1_,wi,100_200_230_250_260_220_240_210,xi,1900_.html'
}

In [5]:
# Taking data from the main page and returning Sets of values for each attribute
def take_data_from_main(response):
  soup = BeautifulSoup(response.content, 'html.parser')
  title_list = soup.find_all(class_ ='list__item__content__title__name link')
  district_list = soup.select('p[class="list__item__content__subtitle"]')
  area_list = soup.select('li[class = "list__item__details__icons__element details--icons--element--powierzchnia"] p[class = "list__item__details__icons__element__desc"]')
  number_of_rooms_list = soup.select('li[class = "list__item__details__icons__element details--icons--element--l_pokoi"] p[class = "list__item__details__icons__element__desc"]')
  year_of_creation_list = soup.select('li[class = "list__item__details__icons__element details--icons--element--rok_budowy"] p[class = "list__item__details__icons__element__desc"]')
  price_list = soup.select('p[class = "list__item__details__info details--info--price"]')
  href_list = soup.select('a[class="list__item__content__title__name link"]')
  return title_list, district_list, area_list, number_of_rooms_list, year_of_creation_list, price_list, href_list

# Diving into the page of an individual advertisement and returning Sets of values for each attribute which was not shown on a main page
def take_data_from_individual(link):
  soup_for_specific_advertisement = BeautifulSoup(requests.get(link).content, 'html.parser')
  type_of_building = soup_for_specific_advertisement.select('div[class="xogField xogField--rodzaj_nieruchomosci"] span[class="xogField__value"]')[0].get_text().strip()
  additional_list = soup_for_specific_advertisement.select('li[class="oglFieldList__item"] b')

  # Adding a try clause because not every offer has an information of the floor (Not given or an offer is a house)
  try:
    floor = soup_for_specific_advertisement.select('a[class="xogField xogField--pietro xogField--withIcon"] span[class="xogField__value xogField__value--big"]')[0].get_text().strip()
  except:
    floor = "nan"
  return type_of_building, additional_list, floor

def get_coordinates(address):
    API_KEY = '727a54b09b424a679c79bba450f60661'
    url = f"https://api.geoapify.com/v1/geocode/search?text={address}&limit=1&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        # Parse the JSON data from the response
        data = response.json()
    
        # Extract the first result from the data
        result = data["features"][0]
    
        # Extract the latitude and longitude of the result
        latitude = result["geometry"]["coordinates"][1]
        longitude = result["geometry"]["coordinates"][0]
    
        return latitude, longitude
    else:
        print(f"Request failed with status code {response.status_code}")
        return None, None


# Searching through the content of the page to gather necessary data such as:
- title
- area of the estate
- number of rooms
- district and street where estate is placed
- year of creation

In [6]:
#Initializing a list which would save records with advertisements data
data = []

#Searching data for secondary market (rynek wtórny)
url = urls['ogloszenia_wtórny']


"""
Iterating through each page and gathering necessary data such as:
- title
- district
- area
- number of rooms
- year of creation
- price in PLN (for m2)
- link of individual adv (to dive further)

"""
for page in range(0,160):
    print("Page number: " + str(page))
    response = requests.get(url, params={'strona': f"{page}"})
    print(response)
    if response.ok :

        # Using previously defined function for taking data from a main page
        title_list, district_list, area_list, number_of_rooms_list, year_of_creation_list, price_list, href_list = take_data_from_main(response)

        print(len(title_list))

        # Iterating through lists to create separate records
        for element in range(len(title_list)):
            try:
              price = price_list[element].get_text().replace(" ", "").rstrip("zł/m2")
            except:
              continue
            tytul = title_list[element].get_text().strip().replace("|", " ")
            district = district_list[element].get_text().strip()
            latitude, longitude = get_coordinates(district + " " + str(10))
            powierzchnia = area_list[element].get_text().strip().rstrip(" m2")
            number_of_rooms = number_of_rooms_list[element].get_text().strip()
            year_of_creation = year_of_creation_list[element].get_text().strip()
            price = price_list[element].get_text().replace(" ", "").rstrip("zł/m2")
            href = href_list[element]
            link = href['href']
            type_of_building, additional_list, floor = take_data_from_individual(link)

            #Converting a Set of values into a list of strings
            additional_list_str = []
            find_string = lambda x : 1 if x.lower() in additional_list_str else 0
            for item in range(len(additional_list)):
              additional_list_str.append(additional_list[item].get_text().strip().lower())

            # Using lambda function to check if an asset for a buyer is in the offer
            garage = find_string('Garaż')
            balcony = find_string('Balkon')
            internet = find_string('Internet')
            parking = find_string('Miejsce Parkingowe')
            elevator = find_string('Winda')
            terrace = find_string('Taras')
            basement = find_string('Piwnica')
            garden = find_string('Ogródek')
            #print("Garage: " + str(garage) + " Balcony: " + str(balcony) + " Internet: " + str(internet) + " Parking: " + str(parking) + " Elevator: " + str(elevator) + " Terrace: " + str(terrace) + " Basement: " + str(basement))

            # Creating a record for a specific offer and adding it to a list
            record = f"{tytul}|{district}|{latitude}|{longitude}|{powierzchnia}|{number_of_rooms}|{year_of_creation}|{price}|{type_of_building}|{garage}|{balcony}|{internet}|{parking}|{elevator}|{floor}|{terrace}|{basement}|{garden}|secondary"
            data.append(record)

Page number: 0
<Response [200]>
30
Page number: 1
<Response [200]>
30
Page number: 2
<Response [200]>
30
Page number: 3
<Response [200]>
30
Page number: 4
<Response [200]>
30
Page number: 5
<Response [200]>
30
Page number: 6
<Response [200]>
30
Page number: 7
<Response [200]>
30
Page number: 8
<Response [200]>
30
Page number: 9
<Response [200]>
30
Page number: 10
<Response [200]>
30
Page number: 11
<Response [200]>
30
Page number: 12
<Response [200]>
30
Page number: 13
<Response [200]>
30
Page number: 14
<Response [200]>
30
Page number: 15
<Response [200]>
30
Page number: 16
<Response [200]>
30
Page number: 17
<Response [200]>
30
Page number: 18
<Response [200]>
30
Page number: 19
<Response [200]>
30
Page number: 20
<Response [200]>
30
Page number: 21
<Response [200]>
30
Page number: 22
<Response [200]>
30
Page number: 23
<Response [200]>
30
Page number: 24
<Response [200]>
30
Page number: 25
<Response [200]>
30
Page number: 26
<Response [200]>
30
Page number: 27
<Response [200]>
30
Pa

In [11]:
# Searching data for primary market (rynek pierwotny)
url = urls['ogloszenia_pierwotny']


"""
Iterating through each page and gathering necessary data such as:
- title
- district
- area
- number of rooms
- year of creation
- price in PLN (for m2)
- link of individual adv (to dive further)

"""
for page in range(0,110):
    print("Page number: " + str(page))
    response = requests.get(url, params={'strona': f"{page}"})
    print(response)
    if response.ok :

        # Using previously defined function for taking data from a main page
        title_list, district_list, area_list, number_of_rooms_list, year_of_creation_list, price_list, href_list = take_data_from_main(response)

        print(len(title_list))

        # Iterating through lists to create separate records
        for element in range(len(title_list)):
            try:
              price = price_list[element].get_text().replace(" ", "").rstrip("zł/m2")
            except:
              continue
            tytul = title_list[element].get_text().strip().replace("|", " ")
            district = district_list[element].get_text().strip()
            latitude, longitude = get_coordinates(district + " " + str(1))
            powierzchnia = area_list[element].get_text().strip().rstrip(" m2")
            number_of_rooms = number_of_rooms_list[element].get_text().strip()
            year_of_creation = year_of_creation_list[element].get_text().strip()
            href = href_list[element]
            link = href['href']
            type_of_building, additional_list, floor = take_data_from_individual(link)

            #Converting a Set of values into a list of strings
            additional_list_str = []
            find_string = lambda x : 1 if x.lower() in additional_list_str else 0
            for item in range(len(additional_list)):
              additional_list_str.append(additional_list[item].get_text().strip().lower())

            # Using lambda function to check if an asset for a buyer is in the offer
            garage = find_string('Garaż')
            balcony = find_string('Balkon')
            internet = find_string('Internet')
            parking = find_string('Miejsce Parkingowe')
            elevator = find_string('Winda')
            terrace = find_string('Taras')
            basement = find_string('Piwnica')
            garden = find_string('Ogródek')

            # Creating a record for a specific offer and adding it to a list. To separate data from each other the '=' sign is being used
            record = f"{tytul}|{district}|{latitude}|{longitude}|{powierzchnia}|{number_of_rooms}|{year_of_creation}|{price}|{type_of_building}|{garage}|{balcony}|{internet}|{parking}|{elevator}|{floor}|{terrace}|{basement}|{garden}|primary"
            data.append(record)

Page number: 0
<Response [200]>
30
Page number: 1
<Response [200]>
30
Page number: 2
<Response [200]>
30
Page number: 3
<Response [200]>
30
Page number: 4
<Response [200]>
30
Page number: 5
<Response [200]>
30
Page number: 6
<Response [200]>
30
Page number: 7
<Response [200]>
30
Page number: 8
<Response [200]>
30
Page number: 9
<Response [200]>
30
Page number: 10
<Response [200]>
30
Page number: 11
<Response [200]>
30
Page number: 12
<Response [200]>
30
Page number: 13
<Response [200]>
30


IndexError: list index out of range

In [8]:
df = pd.DataFrame(data)

In [9]:
print(df)

                                                      0
0     3 pokoje   Nowoczesne   Winda   Balkon|Gdańsk ...
1     4 pokoje / blisko centrum / blisko uczelni|Gda...
2     Klimatyczne   dwupoziomowe   dobrze skomunikow...
3     Apartament Browar Gdański z 60 m2 tarasem|Gdań...
4     Mieszkanie 4 pokojowe (100m2) - przy plaży. Gd...
...                                                 ...
5121  Nowy apartament   4 pokoje   Gdańsk Przymorze|...
5122  Nowy apartament   2 pokoje   Gdańsk Przymorze|...
5123  Super widok, Miejsce w hali, Komórka - Od zara...
5124  3 pokoje na 2025 rok, ogródek - Gdańsk!|Gdańsk...
5125  2 pokoje na 2025 rok, nowoczesny design - Gdań...

[5126 rows x 1 columns]


In [10]:
columns = ["title", "district","latitude", "longitude", "area", "number_of_rooms", "year_of_creation", 'price_for_m2_in_PLN', 'type_of_building', 'garage', 'balcony', 'internet', 'parking', 'elevator', 'floor', 'terrace', 'basement', 'garden','type_of_market']
correct_df = df[0].str.split('|', expand=True)
correct_df.columns = columns
display(correct_df)
correct_df.to_csv("ogloszenia_latitude.csv", sep = '|')

,title,district,latitude,longitude,area,number_of_rooms,year_of_creation,price_for_m2_in_PLN,type_of_building,garage,balcony,internet,parking,elevator,floor,terrace,basement,garden,type_of_market
0,3 pokoje Nowoczesne Winda Balkon,"Gdańsk Łostowice, Niepołomicka",54.31742615,18.593106398457195,50,3,2017,12600,Mieszkanie,0,1,1,1,1,1,0,0,0,secondary
1,4 pokoje / blisko centrum / blisko uczelni,"Gdańsk Suchanino, Powstańców Warszawskich",50.702898,17.907467,64,4,1975,11640,Mieszkanie,0,1,0,0,0,4,0,1,0,secondary
2,Klimatyczne dwupoziomowe dobrze skomunikowane,"Gdynia Wzgórze św. Maksymiliana, Gen. Jarosław...",54.50107075,18.53881452006297,53.44,3,1950,10984,Mieszkanie,0,0,0,0,0,1,0,0,0,secondary
3,Apartament Browar Gdański z 60 m2 tarasem,"Gdańsk Wrzeszcz, Lelewela 6",52.236167,19.363019,60,3,2019,28250,Mieszkanie,1,0,1,0,1,1,0,0,1,secondary
4,Mieszkanie 4 pokojowe (100m2) - przy plaży. Gd...,"Gdańsk Letnica, Starowiejska 81",54.3981198,18.6476667,100,4,2023,14500,Mieszkanie,0,1,1,1,1,5,0,0,0,secondary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,Nowy apartament 4 pokoje Gdańsk Przymorze,"Gdańsk Przymorze, Słupska",54.4038263,18.575947385611514,108.9,4,2024,17631,Mieszkanie,0,0,0,0,0,2,0,0,0,primary
5122,Nowy apartament 2 pokoje Gdańsk Przymorze,"Gdańsk Przymorze, Słupska",54.4038263,18.575947385611514,65.4,2,2024,18649,Mieszkanie,0,0,0,0,0,Parter,0,0,0,primary
5123,"Super widok, Miejsce w hali, Komórka - Od zaraz","Gdańsk Letnica, Sucha",54.39164455,18.64598825,31,1,2023,19323,Mieszkanie,0,1,1,1,1,10,0,0,0,primary
5124,"3 pokoje na 2025 rok, ogródek - Gdańsk!","Gdańsk Łostowice, Pastelowa",52.092517,20.635421,50.97,3,2025,11006,Mieszkanie,0,1,0,1,1,Parter,0,0,0,primary


In [39]:
correct_df['area'].replace('', 0.0, inplace = True)
correct_df['price_for_m2_in_PLN'].replace('', 0.0, inplace = True)
correct_df[['area', 'price_for_m2_in_PLN']] = correct_df[['area', 'price_for_m2_in_PLN']].astype(float)
correct_df[['number_of_rooms', 'year_of_creation']] = correct_df[['number_of_rooms', 'year_of_creation']].astype(int)
correct_df[['garage', 'balcony', 'internet', 'parking', 'elevator', 'terrace', 'basement', 'garden']] = correct_df[['garage', 'balcony', 'internet', 'parking', 'elevator', 'terrace', 'basement', 'garden']].astype(int)
correct_df[['garage', 'balcony', 'internet', 'parking', 'elevator', 'terrace', 'basement', 'garden']] = correct_df[['garage', 'balcony', 'internet', 'parking', 'elevator', 'terrace', 'basement', 'garden']].astype(bool)
correct_df['floor'].replace('nan', np.nan, inplace = True)
correct_df['floor'].replace("Parter", "0", inplace = True)
correct_df['area'].replace(0.0, None ,inplace = True)
correct_df['price_for_m2_in_PLN'].replace(0.0, None ,inplace = True)
print(correct_df.dtypes)

title                  object
district               object
area                   object
number_of_rooms         int32
year_of_creation        int32
price_for_m2_in_PLN    object
type_of_building       object
garage                   bool
balcony                  bool
internet                 bool
parking                  bool
elevator                 bool
floor                  object
terrace                  bool
basement                 bool
garden                   bool
type_of_market         object
dtype: object


C:\Users\kubak\AppData\Local\Temp\ipykernel_25656\3114986829.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  correct_df['area'].replace('', 0.0, inplace = True)
C:\Users\kubak\AppData\Local\Temp\ipykernel_25656\3114986829.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

In [35]:
np.where(pd.isnull(correct_df['price_for_m2_in_PLN']))

(array([2854, 4584], dtype=int64),)

In [36]:
print(df.loc[[662]])

                                                     0
662  2 pokoje   słoneczne mieszkanie   GARAŻ|Gdańsk...


In [50]:
data_from_file = pd.read_csv("ogloszenia.csv", sep='|')
data_from_file.columns.values[0] = 'ID'
display(data_from_file)

,ID,title,district,area,number_of_rooms,year_of_creation,price_for_m2_in_PLN,type_of_building,garage,balcony,internet,parking,elevator,floor,terrace,basement,garden,type_of_market
0,0,3pok 50m2 taras 10m2 blisko morza IIQ24 SE,Gdańsk Letnica,59.00,3,2024,12831.0,Mieszkanie,1,1,0,0,1,1,1,1,0,secondary
1,1,Wysoki standard 4 pok Oliwa kameralne,"Gdańsk Oliwa, Opacka",99.00,4,2020,2424.0,Mieszkanie,0,1,0,1,1,3,0,0,0,secondary
2,2,"Piękne 3 pokoje po remoncie, przy plaży","Gdańsk Jelitkowo, ul. Tysiąclecia",41.07,3,1970,19455.0,Mieszkanie,0,0,0,0,0,4,0,0,0,secondary
3,3,Widok na Jar Wilanowski - blok przy samym Parku,"Gdańsk Chełm, Prof. Romualda Cebertowicza",63.10,3,1989,9984.0,Mieszkanie,0,1,1,0,0,4,0,1,0,secondary
4,4,Energooszczędne Mieszkanie 75m2 - 4 pokoje,Gdańsk,75.00,4,2024,5187.0,Mieszkanie,0,0,0,0,0,Parter,0,0,0,secondary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8002,8002,INPRO S.A. - OPTIMA - mieszkanie 3-pok. 56.33 m2,"Gdynia Dąbrowa, Serdecznikowa",56.33,3,2024,10368.0,Mieszkanie,1,1,1,1,0,2,0,1,0,primary
8003,8003,INPRO S.A. - OPTIMA - mieszkanie 3-pok. 61.44 m2,"Gdynia Dąbrowa, Serdecznikowa",61.44,3,2024,10368.0,Mieszkanie,1,1,1,1,0,3,0,1,0,primary
8004,8004,INPRO S.A. - OPTIMA - mieszkanie 2-pok. 47.22 ...,"Gdynia Dąbrowa, Serdecznikowa",47.00,2,2025,10584.0,Mieszkanie,1,1,1,1,0,1,0,1,0,primary
8005,8005,INPRO S.A. - OPTIMA - mieszkanie 2-pok. 46.30 ...,"Gdynia Dąbrowa, Serdecznikowa",46.30,2,2025,10584.0,Mieszkanie,1,1,1,1,0,1,0,1,0,primary


In [51]:
data_from_file['type_of_building'].unique()

array(['Mieszkanie', 'Dom bliźniak', 'Dom wolnostojący', 'Dom kamienica',
       'Dom szeregowy', 'Dom rekreacyjny', 'Dom piętro domu'],
      dtype=object)

In [52]:
# Checking for values in district column without comma
display(data_from_file.loc[data_from_file['district'].str.contains(",") == False])
# Adding a comma and null value after it to the values without comma
data_from_file.loc[data_from_file['district'].str.contains(",") == False, 'district'] = data_from_file.loc[data_from_file['district'].str.contains(",") == False, 'district'] + ", nan"
display(data_from_file)

,ID,title,district,area,number_of_rooms,year_of_creation,price_for_m2_in_PLN,type_of_building,garage,balcony,internet,parking,elevator,floor,terrace,basement,garden,type_of_market
0,0,3pok 50m2 taras 10m2 blisko morza IIQ24 SE,Gdańsk Letnica,59.00,3,2024,12831.0,Mieszkanie,1,1,0,0,1,1,1,1,0,secondary
4,4,Energooszczędne Mieszkanie 75m2 - 4 pokoje,Gdańsk,75.00,4,2024,5187.0,Mieszkanie,0,0,0,0,0,Parter,0,0,0,secondary
37,37,Projekt architekta* Umeblowane*Balkon+ 2xParking,Sopot Górny Sopot,87.00,4,1910,19253.0,Mieszkanie,0,0,0,0,0,1,0,0,0,secondary
86,86,Prywatne 63 m/3 pok wysoki standard 2020 taras...,Gdańsk Osowa,63.00,3,2020,13794.0,Mieszkanie,0,0,1,1,1,Parter,1,1,1,secondary
97,97,Dom Gdańsk Matarnia (os. Klukowo),Gdańsk Matarnia - Rębiechowo,160.00,4,2008,11250.0,Dom wolnostojący,1,0,1,0,0,NaN,1,0,1,secondary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4770,4770,Dom doskonały pod działalność jak i dla rodziny!,Gdańsk Ujeścisko,148.00,4,2017,11351.0,Dom wolnostojący,1,0,0,0,0,NaN,0,0,0,secondary
4777,4777,Dom 80m od plaży na Kamiennej Górze w Gdyni,Gdynia Kamienna Góra,351.00,8,1993,27920.0,Dom wolnostojący,1,0,0,0,0,NaN,0,0,0,secondary
4779,4779,Dom wolnostojący z garażem gdynia dąbrowa!,Gdynia Dąbrowa,217.00,5,2008,8521.0,Dom wolnostojący,0,0,0,0,0,NaN,0,0,0,secondary
4793,4793,Atrakcyjny dom na Dąbrowie blisko lasu !,Gdynia Dąbrowa,217.09,5,2001,8706.0,Dom wolnostojący,1,1,1,1,0,NaN,0,0,1,secondary


,ID,title,district,area,number_of_rooms,year_of_creation,price_for_m2_in_PLN,type_of_building,garage,balcony,internet,parking,elevator,floor,terrace,basement,garden,type_of_market
0,0,3pok 50m2 taras 10m2 blisko morza IIQ24 SE,"Gdańsk Letnica, nan",59.00,3,2024,12831.0,Mieszkanie,1,1,0,0,1,1,1,1,0,secondary
1,1,Wysoki standard 4 pok Oliwa kameralne,"Gdańsk Oliwa, Opacka",99.00,4,2020,2424.0,Mieszkanie,0,1,0,1,1,3,0,0,0,secondary
2,2,"Piękne 3 pokoje po remoncie, przy plaży","Gdańsk Jelitkowo, ul. Tysiąclecia",41.07,3,1970,19455.0,Mieszkanie,0,0,0,0,0,4,0,0,0,secondary
3,3,Widok na Jar Wilanowski - blok przy samym Parku,"Gdańsk Chełm, Prof. Romualda Cebertowicza",63.10,3,1989,9984.0,Mieszkanie,0,1,1,0,0,4,0,1,0,secondary
4,4,Energooszczędne Mieszkanie 75m2 - 4 pokoje,"Gdańsk, nan",75.00,4,2024,5187.0,Mieszkanie,0,0,0,0,0,Parter,0,0,0,secondary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8002,8002,INPRO S.A. - OPTIMA - mieszkanie 3-pok. 56.33 m2,"Gdynia Dąbrowa, Serdecznikowa",56.33,3,2024,10368.0,Mieszkanie,1,1,1,1,0,2,0,1,0,primary
8003,8003,INPRO S.A. - OPTIMA - mieszkanie 3-pok. 61.44 m2,"Gdynia Dąbrowa, Serdecznikowa",61.44,3,2024,10368.0,Mieszkanie,1,1,1,1,0,3,0,1,0,primary
8004,8004,INPRO S.A. - OPTIMA - mieszkanie 2-pok. 47.22 ...,"Gdynia Dąbrowa, Serdecznikowa",47.00,2,2025,10584.0,Mieszkanie,1,1,1,1,0,1,0,1,0,primary
8005,8005,INPRO S.A. - OPTIMA - mieszkanie 2-pok. 46.30 ...,"Gdynia Dąbrowa, Serdecznikowa",46.30,2,2025,10584.0,Mieszkanie,1,1,1,1,0,1,0,1,0,primary


In [53]:
data_from_file[['district', 'street']] = data_from_file['district'].str.split(",", expand = True)
display(data_from_file)

,ID,title,district,area,number_of_rooms,year_of_creation,price_for_m2_in_PLN,type_of_building,garage,balcony,internet,parking,elevator,floor,terrace,basement,garden,type_of_market,street
0,0,3pok 50m2 taras 10m2 blisko morza IIQ24 SE,Gdańsk Letnica,59.00,3,2024,12831.0,Mieszkanie,1,1,0,0,1,1,1,1,0,secondary,nan
1,1,Wysoki standard 4 pok Oliwa kameralne,Gdańsk Oliwa,99.00,4,2020,2424.0,Mieszkanie,0,1,0,1,1,3,0,0,0,secondary,Opacka
2,2,"Piękne 3 pokoje po remoncie, przy plaży",Gdańsk Jelitkowo,41.07,3,1970,19455.0,Mieszkanie,0,0,0,0,0,4,0,0,0,secondary,ul. Tysiąclecia
3,3,Widok na Jar Wilanowski - blok przy samym Parku,Gdańsk Chełm,63.10,3,1989,9984.0,Mieszkanie,0,1,1,0,0,4,0,1,0,secondary,Prof. Romualda Cebertowicza
4,4,Energooszczędne Mieszkanie 75m2 - 4 pokoje,Gdańsk,75.00,4,2024,5187.0,Mieszkanie,0,0,0,0,0,Parter,0,0,0,secondary,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8002,8002,INPRO S.A. - OPTIMA - mieszkanie 3-pok. 56.33 m2,Gdynia Dąbrowa,56.33,3,2024,10368.0,Mieszkanie,1,1,1,1,0,2,0,1,0,primary,Serdecznikowa
8003,8003,INPRO S.A. - OPTIMA - mieszkanie 3-pok. 61.44 m2,Gdynia Dąbrowa,61.44,3,2024,10368.0,Mieszkanie,1,1,1,1,0,3,0,1,0,primary,Serdecznikowa
8004,8004,INPRO S.A. - OPTIMA - mieszkanie 2-pok. 47.22 ...,Gdynia Dąbrowa,47.00,2,2025,10584.0,Mieszkanie,1,1,1,1,0,1,0,1,0,primary,Serdecznikowa
8005,8005,INPRO S.A. - OPTIMA - mieszkanie 2-pok. 46.30 ...,Gdynia Dąbrowa,46.30,2,2025,10584.0,Mieszkanie,1,1,1,1,0,1,0,1,0,primary,Serdecznikowa


In [55]:
data_from_file.loc[data_from_file['district'].str.contains(" ") == False]

,ID,title,district,area,number_of_rooms,year_of_creation,price_for_m2_in_PLN,type_of_building,garage,balcony,internet,parking,elevator,floor,terrace,basement,garden,type_of_market,street
4,4,Energooszczędne Mieszkanie 75m2 - 4 pokoje,Gdańsk,75.00,4,2024,5187.0,Mieszkanie,0,0,0,0,0,Parter,0,0,0,secondary,nan
119,119,Sopot 2-pokojowe sprzedam,Sopot,36.00,2,1975,18194.0,Mieszkanie,0,0,0,0,0,5,0,0,0,secondary,Niepodległości 708
209,209,Atrakcyjna lokalizacja centrum,Gdynia,50.80,3,1926,13287.0,Mieszkanie,0,0,1,0,0,Parter,0,0,0,secondary,ŚLĄSKA 9
845,845,Dom Bliżniak 6 pokoi,Gdańsk,130.00,4,2023,6077.0,Mieszkanie,1,0,0,1,0,Parter,0,0,1,secondary,nan
947,947,Sopot Dolny - sprzedam mieszkanie,Sopot,4.00,2,1900,2845.0,Mieszkanie,0,0,0,0,0,1,0,0,0,secondary,Fryderyka Chopina
1007,1007,Nowe mieszkanie dwa pokoje + strych - Trąbki W...,Gdańsk,55.00,2,2022,6636.0,Mieszkanie,0,0,0,0,0,2,0,0,0,secondary,nan
1142,1142,Dom w lesie na Kaszubach,Gdynia,86.00,3,2024,694.0,Dom wolnostojący,0,0,0,0,0,NaN,0,0,0,secondary,nan
1284,1284,Komfortowe Mieszkanie z systemem Smart Home,Gdańsk,68.60,3,2009,10029.0,Mieszkanie,0,0,0,0,0,2,0,0,0,secondary,nan
1447,1447,BRA BANK 70 metrów apartament do własnej aranż...,Gdańsk,70.00,3,2021,26429.0,Mieszkanie,1,0,0,0,1,6,0,0,0,secondary,nan
1635,1635,Mieszkanie na sprzedaż Al. Niepodległości,Sopot,50.00,3,1950,12980.0,Mieszkanie,0,0,0,0,0,Parter,0,0,0,secondary,nan
